In [7]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import random
import ipywidgets as widgets

In [8]:
###

# Consumption Behaviour
## income, price of energy set, technology and household needs
# Income changes over time 
## economic growth
# Population changes 
## population growth or decline? 
# Policy regulations
## IMPORTANT BUT CURRENTLY NOT SPECIFIED 
# Energy efficiency (technology)

In [9]:
# ## I HAVE LOADED THE APROPRIATE PACKAGES 

# ## FIGURE SETTINGS 

# yummy_figs = [10,6]
# # Define parameters
# num_people = 1000
# median_income = 50000
# gini = 0.25
# gr = [-0.05,0.05]
# periods = 10

class GiniCalculationError(Exception):
    pass

def calculate_gini(incomes):
    # Calculate Gini coefficient for a list of incomes
    incomes = np.sort(incomes)
    n = len(incomes)
    index = np.arange(1, n + 1)
    return ((np.sum((2 * index - n  - 1) * incomes)) / (n * np.sum(incomes)))

def generate_income_distribution(num_people, median_income, gini_target):
    alpha = (gini_target + 1) / (2 - gini_target)  # Set initial alpha
    for _ in range(10000):  # Limit the number of iterations
        incomes = stats.gamma.rvs(alpha, scale=median_income/alpha, size=num_people)  # Generate a random income distribution
        gini_current = calculate_gini(incomes)  # Calculate the current Gini coefficient
        if np.isclose(gini_current, gini_target, atol=0.01):  # Check if the current Gini coefficient is close to the target
            return incomes
        elif gini_current < gini_target:  # If the current Gini coefficient is too low, decrease alpha
            alpha *= 0.9
        else:  # If the current Gini coefficient is too high, increase alpha
            alpha *= 1.1

    # If we've reached this point, the desired Gini coefficient was not reached
    error_message = f"Failed to reach target Gini coefficient in 1000 iterations. Current Gini: {gini_current}"
    raise GiniCalculationError(error_message)

def calculate_wealth(incomes, periods):
    # Initialize lists to hold the wealth and Gini coefficients at each time step
    wealth_over_time = []
    gini_over_time = []
    wealth_distribution_over_time = []
    
    # Calculate the wealth for each time period
    for _ in range(periods):
        # Calculate the wealth by summing the incomes
        wealth = np.sum(incomes)
        
        # Add the wealth to the list
        wealth_over_time.append(wealth)
        
        # Calculate the Gini coefficient for the current income distribution
        gini = calculate_gini(incomes)
        
        # Add the Gini coefficient to the list
        gini_over_time.append(gini)
        
        # Calculate the wealth distribution for the current income distribution
        wealth_distribution = calculate_wealth_distribution(incomes)
        
        # Add the wealth distribution to the list
        wealth_distribution_over_time.append(wealth_distribution)
        
        # Set a random growth rate for the current period
        growth_rate = np.random.uniform(gr[0], gr[1])
        
        # Increase the incomes by the growth rate
        incomes *= (1 + growth_rate)
    
    return wealth_over_time, gini_over_time, wealth_distribution_over_time


def calculate_wealth_distribution(incomes, num_bins=10):
    # Calculate the total wealth
    total_wealth = np.sum(incomes)
    
    # Calculate the histogram of wealth
    hist, bin_edges = np.histogram(incomes, bins=num_bins)
    
    # Calculate the wealth in each bin
    bin_wealths = [(bin_edges[i+1] - bin_edges[i]) * hist[i] for i in range(num_bins)]
    
    # Normalize the wealths by total wealth to get wealth distribution
    wealth_distribution = bin_wealths / total_wealth
    
    return wealth_distribution

# # Generate the income distribution
# incomes = generate_income_distribution(num_people, median_income, gini)

# # Calculate the wealth distribution
# wealth_distribution = calculate_wealth_distribution(incomes)

# # Plot the wealth over time
# plot_wealth_over_time(wealth_over_time)

# # Plot the wealth distribution
# plot_wealth_distribution(wealth_distribution)

# plot_lorenz_curve(incomes)



# MODEL

In [15]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
from IPython.display import clear_output
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import random
import ipywidgets as widgets


## HELPER FUNCTIONS 

class GiniCalculationError(Exception):
    pass

def calculate_gini(incomes):
    # Calculate Gini coefficient for a list of incomes
    incomes = np.sort(incomes)
    n = len(incomes)
    index = np.arange(1, n + 1)
    return ((np.sum((2 * index - n  - 1) * incomes)) / (n * np.sum(incomes)))

def generate_income_distribution(num_people, median_income, gini_target):
    alpha = (gini_target + 1) / (2 - gini_target)  # Set initial alpha
    for _ in range(10000):  # Limit the number of iterations
        incomes = stats.gamma.rvs(alpha, scale=median_income/alpha, size=num_people)  # Generate a random income distribution
        gini_current = calculate_gini(incomes)  # Calculate the current Gini coefficient
        if np.isclose(gini_current, gini_target, atol=0.01):  # Check if the current Gini coefficient is close to the target
            return incomes
        elif gini_current < gini_target:  # If the current Gini coefficient is too low, decrease alpha
            alpha *= 0.9
        else:  # If the current Gini coefficient is too high, increase alpha
            alpha *= 1.1

    # If we've reached this point, the desired Gini coefficient was not reached
    error_message = f"Failed to reach target Gini coefficient in 1000 iterations. Current Gini: {gini_current}"
    raise GiniCalculationError(error_message)

def calculate_wealth(incomes, periods):
    # Initialize lists to hold the wealth and Gini coefficients at each time step
    wealth_over_time = []
    gini_over_time = []
    wealth_distribution_over_time = []
    
    # Calculate the wealth for each time period
    for _ in range(periods):
        # Calculate the wealth by summing the incomes
        wealth = np.sum(incomes)
        
        # Add the wealth to the list
        wealth_over_time.append(wealth)
        
        # Calculate the Gini coefficient for the current income distribution
        gini = calculate_gini(incomes)
        
        # Add the Gini coefficient to the list
        gini_over_time.append(gini)
        
        # Calculate the wealth distribution for the current income distribution
        wealth_distribution = calculate_wealth_distribution(incomes)
        
        # Add the wealth distribution to the list
        wealth_distribution_over_time.append(wealth_distribution)
        
        # Set a random growth rate for the current period
        growth_rate = np.random.uniform(gr[0], gr[1])
        
        # Increase the incomes by the growth rate
        incomes *= (1 + growth_rate)
    
    return wealth_over_time, gini_over_time, wealth_distribution_over_time


def calculate_wealth_distribution(incomes, num_bins=10):
    # Calculate the total wealth
    total_wealth = np.sum(incomes)
    
    # Calculate the histogram of wealth
    hist, bin_edges = np.histogram(incomes, bins=num_bins)
    
    # Calculate the wealth in each bin
    bin_wealths = [(bin_edges[i+1] - bin_edges[i]) * hist[i] for i in range(num_bins)]
    
    # Normalize the wealths by total wealth to get wealth distribution
    wealth_distribution = bin_wealths / total_wealth
    
    return wealth_distribution


# PARAMETERS
yummy_figs = [10,6]
num_people = 1000
median_income = 50000
gini = 0.25
gr = [10,20]
periods = 10


class Person(Agent):
    def __init__(self, unique_id, model, initial_income):
        super().__init__(unique_id, model)
        self.wealth = initial_income
    
    def step(self):
        growth_rate = self.random.uniform(gr[0], gr[1])
        self.wealth *= (1 + growth_rate)

class Economy(Model):
    def __init__(self, num_people, median_income, gini_target):
        self.num_people = num_people
        self.schedule = RandomActivation(self)
        incomes = generate_income_distribution(num_people, median_income, gini_target)
        for i in range(num_people):
            a = Person(i, self, incomes[i])
            self.schedule.add(a)
        
        self.datacollector = DataCollector(
            model_reporters={"Total Wealth": lambda m: sum([agent.wealth for agent in m.schedule.agents]), 
                     "Gini": lambda m: calculate_gini([agent.wealth for agent in m.schedule.agents]),
                     "Wealth Distribution": lambda m: calculate_wealth_distribution([agent.wealth for agent in m.schedule.agents])},
            agent_reporters={"Wealth": lambda a: a.wealth})

    def step(self):
        '''Advance the model by one step.'''
        self.datacollector.collect(self)
        self.schedule.step()

# Instantiate and run model
model = Economy(1000, 50000, gini)
for i in range(periods):
    model.step()

# Access the collected data
model_df = model.datacollector.get_model_vars_dataframe()
agent_df = model.datacollector.get_agent_vars_dataframe()



In [16]:
## PLOTS 

def plot_wealth_over_time(wealth_over_time):
    plt.figure(figsize=(10,6))
    plt.plot(wealth_over_time)
    plt.xlabel('Time')
    plt.ylabel('Wealth')
    plt.title('Total wealth over time')
    plt.show()

def plot_wealth_distribution(wealth_distribution):
    plt.figure(figsize=(10,6))
    plt.bar(range(1, 11), wealth_distribution)
    plt.xlabel('Quintile')
    plt.ylabel('Wealth')
    plt.title('Wealth distribution')
    plt.show()

def plot_lorenz_curve(X, yummy_figs = yummy_figs):

    X_sorted = np.sort(X)  # Sort the incomes
    lorenz = np.cumsum(X_sorted) / np.sum(X_sorted)
    lorenz = np.insert(lorenz, 0, 0)
    lorenz[0], lorenz[-1]

    fig, ax = plt.subplots(figsize=[yummy_figs[0],yummy_figs[1]])
    ax.scatter(np.arange(lorenz.size)/(lorenz.size-1), lorenz, 
            marker='.', color='darkgreen', s=100)
    ## line plot of equality
    plt.title("Lorenz Curve")
    ax.plot([0,1], [0,1], color='k')
    plt.show()

In [17]:
@widgets.interact
def interactive_plots(t=widgets.IntSlider(min=0, max=model.schedule.steps-1, step=1, value=0)):
    time_step = t
    clear_output(wait=True)
    wealth_dist = model_df.loc[time_step, "Wealth Distribution"]
    total_wealth = model_df.loc[time_step, "Total Wealth"]
    gini_data =  model_df.loc[time_step, "Gini"]
    incomes = agent_df.loc[time_step, "Wealth"]
    
    # Plot the wealth distribution
    plot_wealth_distribution(wealth_dist)
    
    # Plot the Lorenz curve
    plot_lorenz_curve(incomes)

interactive(children=(IntSlider(value=0, description='t', max=9), Output()), _dom_classes=('widget-interact',)…